# Week 7 Seminar Exercises  
## Some useful notes from the lab  

### Error Detection with Parity Bits  

Single Parity Check: Adds a parity bit (even or odd) to detect single-bit errors.  
2D Parity Check: Adds row and column parity bits to detect and sometimes correct errors.  

### Checksum Using One’s Complement

Process: Sum data words with end-around carry, take complement for checksum.
Verification: Sum data + checksum; valid if result is all s (example:  0xFFFF for 16-bit).

### Slotted ALOHA Protocol

Efficiency:  It peaks at 1/e ≈ 0.37 when transmission probability equals with p = 1/N.  
### Why Efficiency Peaks at ~37%
Slotted ALOHA achieves maximum efficiency when the transmission probability \( p = \frac{1}{N} \), where \( N \) is the number of nodes. This balances  the collisions and idle slots, maximizing successful transmissions.  


In [ ]:
# Exercise 1: Parity Bit Implementation
# Objective: Implement even parity and simulate error detection.

def compute_even_parity(data):
    return sum(data) % 2

# Example
data = [1, 0, 1, 0, 1, 1, 0, 0]
parity_bit = compute_even_parity(data)
transmitted_data = data + [parity_bit]

# Introduce error at index 3
error_index = 3
data_with_error = transmitted_data.copy()
data_with_error[error_index] = 1 - data_with_error[error_index]

# Check for errors
if sum(data_with_error) % 2 == 0:
    print("No error detected")
else:
    print("Error detected")

In [ ]:
# Exercise 2: 2D Parity Check
# Objective: Detect and correct single-bit errors using row/column parity.

import numpy as np

def compute_parity(mat):
    return np.sum(mat, axis=1) % 2, np.sum(mat, axis=0) % 2

# Original data
data = np.array([
    [1, 0, 1, 1],
    [0, 1, 0, 0],
    [1, 1, 1, 0],
    [0, 0, 1, 1]
])
row_par, col_par = compute_parity(data)

# Introduce error at (2,1)
data_err = data.copy()
data_err[2, 1] = 1 - data_err[2, 1]
new_row_par, new_col_par = compute_parity(data_err)

# Detect and correct
err_row = np.where(new_row_par != row_par)[0]
err_col = np.where(new_col_par != col_par)[0]
if err_row.size == 1 and err_col.size == 1:
    data_err[err_row[0], err_col[0]] = 1 - data_err[err_row[0], err_col[0]]

if err_row.size == 1 and err_col.size == 1:
    error_location = (err_row[0], err_col[0])
    print(f"Error detected at: {error_location}")
    # Correct the error by flipping the bit
    data_err[error_location] = 1 - data_err[error_location]
    print("Corrected Data:\n", data_err)
else:
    print("No single-bit error detected.")

In [ ]:
# Exercise 3: One’s Complement Checksum
# Objective: Compute and verify checksums for error detection.

def ones_complement_sum(a, b, bit_size=16):
    result = a + b
    if result >= (1 << bit_size):
        result = (result + 1) & ((1 << bit_size) - 1)
    return result

def calculate_checksum(data, bit_size=16):
    checksum = 0
    for word in data:
        checksum = ones_complement_sum(checksum, word, bit_size)
    return ~checksum & ((1 << bit_size) - 1)

def verify_checksum(data, checksum, bit_size=16):
    total = 0
    for word in data:
        total = ones_complement_sum(total, word, bit_size)
    total = ones_complement_sum(total, checksum, bit_size)
    return total == (1 << bit_size) - 1

# Example
data = [0b1010101010101010, 0b1100110011001100, 0b1111000011110000]
checksum = calculate_checksum(data)
print(f"Valid: {verify_checksum(data, checksum)}")

In [ ]:
# Exercise 4: Slotted ALOHA Simulation
# Objective: Model Slotted ALOHA efficiency vs. transmission probability.
import random
import matplotlib.pyplot as plt
import numpy as np

def simulate_slotted_aloha(n_nodes, p, n_slots):
    successes = 0
    for _ in range(n_slots):
        transmissions = sum(1 for _ in range(n_nodes) if random.random() < p)
        if transmissions == 1:
            successes += 1
    return successes / n_slots

n_nodes = 50
ps = np.linspace(0, 1, 50)
efficiencies = [simulate_slotted_aloha(n_nodes, p, 10000) for p in ps]

plt.plot(ps, efficiencies, marker='o')
plt.xlabel('Transmission Probability')
plt.ylabel('Efficiency')
plt.title('Slotted ALOHA Efficiency')
plt.show()